In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

2.10.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8612534666032900081
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5744099328
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13034587899105868184
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [2]:
import os
import json
import csv

dataset_path = r'E:\AInotes\자세교정\전처리\RNN데이터 전처리\sequence_label'
# JSON 파일에서 데이터 불러오기

sequence_data = []
label_data = []

for roots, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(roots, file)
            with open(file_path, "r") as json_file:
                loaded_data = json.load(json_file)

                # 불러온 데이터 확인
                sequences_loaded = loaded_data["sequences"]
                labels_loaded = loaded_data["labels"]

                # RNN에 사용할 데이터 형식으로 extend 저장
                sequence_data.extend(sequences_loaded)
                label_data.extend(labels_loaded)
                # 시퀀스의 개수 확인
                num_sequences = len(sequences_loaded)

num_sequence_data = len(sequence_data)

print("시퀀스의 개수:", num_sequence_data)
print("라벨의 개수:", len(label_data))


시퀀스의 개수: 13143
라벨의 개수: 13143


In [2]:
import numpy as np

def pad_sequences_custom(sequences, maxlen=None, dtype='float32', padding='post', value=0.0):
    # 각 시퀀스의 길이를 기반으로 최대 길이를 결정
    if maxlen is None:
        maxlen = max(len(seq) for seq in sequences)
    
    # 신체 키포인트 데이터의 차원을 고려하여 결과 배열 초기화
    # 예: 각 키포인트 데이터가 (x, y) 좌표를 가진다고 가정하면, 차원은 2가 됩니다.
    sample_shape = np.asarray(sequences[0]).shape[1:]
    result_shape = (len(sequences), maxlen) + sample_shape
    result = np.full(result_shape, value, dtype=dtype)
    
    # 각 시퀀스를 결과 배열에 복사
    for i, seq in enumerate(sequences):
        if len(seq) > maxlen:  # 시퀀스가 maxlen보다 길 경우, 잘라냅니다.
            seq = seq[:maxlen]
        if padding == 'post':
            result[i, :len(seq)] = seq
        elif padding == 'pre':
            result[i, -len(seq):] = seq
    
    return result

# 사용 예시
# sequence_data는 신체 키포인트 데이터를 포함하는 리스트입니다.
sequence_data_padded = pad_sequences_custom(sequence_data)
sequence_data_np = np.array(sequence_data_padded)

# 이제 sequence_data_np를 모델 학습에 사용할 수 있습니다.


In [5]:
import numpy as np

# 원래 레이블 데이터 예시
label_data = np.array(label_data)  # 여기에 실제 레이블 데이터를 넣으세요.

# 원래 레이블 범위를 연속적인 정수 범위로 매핑하는 사전 생성
unique_labels = np.unique(label_data)
label_mapping = {label: i for i, label in enumerate(unique_labels)}

# 레이블 데이터를 매핑된 정수로 변환
mapped_labels = np.array([label_mapping[label] for label in label_data])
print(len(np.unique(mapped_labels)))
# 이제 mapped_labels를 사용하여 모델을 학습시킬 수 있습니다.


344


In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# 변환된 레이블 데이터를 원-핫 인코딩으로 변환
num_classes = len(np.unique(mapped_labels))  # 클래스 수 계산
labels_one_hot = to_categorical(mapped_labels, num_classes=num_classes)

# 모델 구성
model = Sequential([
    LSTM(64, input_shape=(sequence_data_np.shape[1], sequence_data_np.shape[2])),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

# 모델 학습
history = model.fit(sequence_data_np, labels_one_hot, epochs=30, batch_size=32, validation_split=0.2)

# (선택적) 모델 평가
# 만약 테스트 데이터셋이 있다면, 아래와 같이 모델을 평가할 수 있습니다.
# test_sequence_data_np = ... # 테스트 시퀀스 데이터에 대한 패딩 처리된 np.array
# test_mapped_labels = ... # 테스트 레이블 데이터를 같은 매핑 사전으로 변환한 결과
# test_labels_one_hot = to_categorical(test_mapped_labels, num_classes=num_classes)
# evaluation = model.evaluate(test_sequence_data_np, test_labels_one_hot)
# print("Test Loss:", evaluation[0], "Test Accuracy:", evaluation[1])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                26368     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 344)               22360     
                                                                 
Total params: 52888 (206.59 KB)
Trainable params: 52888 (206.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
329/329